# BINF TP3 - Algorithmes d'alignement par paire

Dans ce TP nous allons manipuler les algorithmes d'alignement par paire.

# Exercice 0 - Echauffement

Q1. Donnez le score de la superposition :

|       |       |
| :---: | :---: |
x       | ATGTCATGA---TAC |
y       | AT--CTAAATGTTAC |


étant donne le schéma d'évaluation :

|       | A     | T     | G     | C     |
| :---: | :---: | :---: | :---: | :---: |
| **A** | 1     | -1    | -1    | -1    |
| **T** | -1    | 1     | -1    | -1    |
| **G** | -1    | -1    | 1     | -1    |
| **C** | -1    | -1    | -1    | 1     |

et

$\gamma(g) = 0.5 |g| + 0.5$

```markdown
1 + 1 + 1 - 1 - 1 - 1 + 1 + 1 + 1 + 1 = 4
y(2) = 0.5 * 2 + 0.5 = 1.5
y(3) = 0.5 * 3 + 0.5 = 2

4 - (1,5 + 2) = 0,5

```

Q2. Alignez les séquences suivantes avec l'algorithme de Levenshtein :  x = ATG et y = ACTG.


|       |   0     | A       | C       | T       | G     |  
| :---: |  :---:  |  :---:  | :---:   | :---:   | :---: |
| **0** |    0    |   1     |    2    |    3    |    4  |
| **A** |    1    |   0     |    1    |    2    |    3  |
| **T** |    2    |   1     |    1    |    1    |    2  |
| **G** |    3    |   2     |    2    |    2    |    1  |

|       |   0     | A       | C       | T       | G     |
| :---: |  :---:  |  :---:  | :---:   | :---:   | :---: |
| **0** |    0    |   _     |    _    |    _    |    _  |
| **A** |    l    |   \     |    _    |    _    |    _  |
| **T** |    l    |   l     |    \    |    \    |    _  |
| **G** |    l    |   l     |    \    |    \    |    \  |

```markdown
x = A_TG
y = ACTG
```

Q3.	Alignez les séquences suivantes avec l'algorithme de Needleman-Wunsch global x = TAT et y = ATGAC en considérant le schéma d'évaluation suivant

|       | A       | T       | G       | C     |
| :---: | :---:   | :---:   | :---:   | :---: |
| **A** | 1       | -0.5    | -0.5    | -0.5  |
| **T** | -0.5    | 1       | -0.5    | -0.5  |
| **G** | -0.5    | -0.5    | 1       | -0.5  |
| **C** | -0.5    | -0.5    | -0.5    | 1     |

et

$\gamma(g) = 0.5 |g|$


|       |0    | A       | T       | G       |A      | C     |
| :---: |:---:| :---:   | :---:   | :---:   | :---: | :---: |
| **0** |0    | -0.5    | -1      | -1.5    |  -2   | -2.5  |
| **T** |-0.5 | -0.5    |  0.5    |   0     | -0.5  | -1    |
| **A** |-1   |  0.5    |   0     |   0     |  1    |  0.5  |
| **T** |-1.5 |   0     |  -1.5   |   1     |  0.5  |  0.5  |

|       |0    | A       | T       | G       |A      | C     |
| :---: |:---:| :---:   | :---:   | :---:   | :---: | :---: |
| **0** |0    | _       | _       | _       |  _    | _     |
| **T** |l    | \       |  \      |   _     | _     | _     |
| **A** |l    |  \      |   _     |   \     |  \    |  _    |
| **T** |l    |   l     |  \      |   _     |  _    |  \    |

```markdown
x = _T_AT
y = ATGAC
```

Q4. Alignez les séquences suivantes avec l'algorithme de Smith-Waterman x = TTGG y = ATGAC en utilisant le schéma d'évaluation de la question précédente.


```markdown
Votre réponse ici
```

# Exercice 1 : Algorithme de Levenshtein - version récursive

Q1. Ecrivez une fonction

levenshtein(x: str, y: str) -> int

qui retourne la distance de Levenshtein entre les séquences x et y en utilisant la  version récursive de l'algorithme.

In [ ]:
def levenshtein(x: str, y: str) -> int:
    if not x:
        return len(y)
    if not y:
        return len(x)

    cost = 0 if x[-1] == y[-1] else 1

    return min(
        levenshtein(x[:-1], y) + 1,
        levenshtein(x, y[:-1]) + 1,
        levenshtein(x[:-1], y[:-1]) + cost
    )

print(levenshtein("CCGT", "CGTCA"))


3


Q2. Vous pouvez tester votre code sur les exemples suivants:


*   $L('CCAG', 'CA') = 2$
*   $L('CCGT', 'CGTCA') = 3$
*   $L(AY678264^*, OQ870305^*) = 310$

$^*$ ids genbank de deux sequences.

# Exercice 2 : Algorithme de Smith-Waterman - version itérative

Q1. Ecrivez la fonction

sw_fwd(x: str, y: str, cmap: dict, sigma: array, (go, ge): list) -> (array, array)

qui construit les matrices $S$ et $B$ en utilisant l'algorithme de Smith-Waterman pour aligner les séquences x et y suivant le schéma d'évaluation donné par la matrice de substitution $\Sigma$ et la fonction d'évaluation des trous $\gamma(n)= g_o + g_e \times n$. Le dictionnaire cmap donne la position des différents nucléotides dans la matrice $\Sigma$. La fonction retourne la paire de matrices de score $S$ et de retour $B$.

In [6]:
import numpy as np

def sw_fwd(x: str, y: str, cmap: dict, sigma: np.array, gap_params: tuple) -> tuple:
    DIAG, UP, LEFT = 1, 2, 3
    m, n = len(x), len(y)
    go, ge = gap_params

    S = np.zeros((m + 1, n + 1))
    B = np.zeros((m + 1, n + 1), dtype=int)

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match_score = S[i-1, j-1] + sigma[cmap[x[i-1]], cmap[y[j-1]]]

            max_del, best_k_del = -np.inf, 0
            for k in range(1, i+1):
                current = S[i-k, j] - (go + ge*(k-1))
                if current > max_del:
                    max_del = current
                    best_k_del = k

            max_ins, best_k_ins = -np.inf, 0
            for k in range(1, j+1):
                current = S[i, j-k] - (go + ge*(k-1))
                if current > max_ins:
                    max_ins = current
                    best_k_ins = k

            options = [(0, 0), (match_score, DIAG), (max_del, UP), (max_ins, LEFT)]
            S[i,j], B[i,j] = max(options, key=lambda x: x[0])

    return S, B


dna_cmap = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
sigma_matrix = np.array([[2, -1, -1, -1],
                          [-1, 2, -1, -1],
                          [-1, -1, 2, -1],
                          [-1, -1, -1, 2]])

go, ge = -2, -1
x_seq = "AGT"
y_seq = "GTT"

S, B = sw_fwd(x_seq, y_seq, dna_cmap, sigma_matrix, (go, ge))
print(S)
print(B)


[[ 0.  0.  0.  0.]
 [ 0.  2.  4.  6.]
 [ 0.  4.  6.  8.]
 [ 0.  6.  8. 10.]]
[[0 0 0 0]
 [0 2 3 3]
 [0 2 2 2]
 [0 2 2 2]]


Q2. Ecrivez la fonction

sw_bwd(x: str, y: str, S: array, B: array) -> (str, str, float)

qui effectue l'etape de retour de l'algorithme de Smith-Waterman etant donné les séquences $x$ et $y$ et les matrices de score $S$ et de retour $B$. La fonction retourne un tuple contenant les alignements des séquences x et y et le score de l'alignement.

In [7]:
import numpy as np

def sw_bwd(x: str, y: str, S: np.ndarray, B: np.ndarray) -> (str, str, float):
    # Initialiser les alignements et le score
    align_x = []
    align_y = []

    # Trouver la position du score maximum dans la matrice S (c'est ici que l'alignement commence)
    i, j = np.unravel_index(np.argmax(S), S.shape)
    score = S[i, j]

    # Effectuer le traceback
    while S[i, j] > 0:
        if B[i, j] == 1:  # Diagonale
            align_x.append(x[i-1])
            align_y.append(y[j-1])
            i -= 1
            j -= 1
        elif B[i, j] == 2:  # Haut
            align_x.append(x[i-1])
            align_y.append('-')
            i -= 1
        elif B[i, j] == 3:  # Gauche
            align_x.append('-')
            align_y.append(y[j-1])
            j -= 1

    # Inverser les alignements car on a commencé à partir de la fin
    align_x.reverse()
    align_y.reverse()

    # Convertir les listes en chaînes
    align_x_str = ''.join(align_x)
    align_y_str = ''.join(align_y)

    return align_x_str, align_y_str, score


Q3. Vous pouvez tester votre code en utilisant le schéma d'évaluation suivant :

In [2]:
cmap = {"A": 0, "T": 1, "G": 2, "C": 3}
m = np.array([[1, -0.5, -0.5, -0.5],
              [-0.5, 1, -0.5, -0.5],
              [-0.5, -0.5, 1, -0.5],
              [-0.5, -0.5, -0.5, 1]])
go = 0
ge = 0.5

*   $SW('TCGC', 'CTTAG')$ retourne un score de $1.5$ à la position $(3,5)$ et l'alignement

In [5]:
from IPython.display import HTML
HTML("<table align='left' style='font-family:Courier New'><tr><th>x:</th><th>TCG</th></tr><tr><th>y:</th><th>TAG</th></tr></table>")

x:,TCG
y:,TAG


*   $SW(AY678264^*, OQ870305^*)$ retourne un score de $342.1$ à la position $(708,717)$ et l'alignement

In [4]:
from IPython.display import HTML
HTML("<table width='300px' align='left' style='font-family:Courier New'><tr><th>x:</th><th nowrap='nowrap'>ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGC-A-CATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAG---GGCGAGGGCGAGGGC--CGCC-CCTACGAGGGCACCCAGACCGC-CAAGCTGAAGGTG-ACCA-AGG---G-TGGCC---CCCT-GCCCTTCGCCT-GGGA-CATCCTGTCC--C--C-T-CAGTTCATGT-A-CGGCT-CCAAGGCCTACGTG-A--AGCAC--C--C--C--G-CCGACATCCCCG-A--CTAC-T--TGAAGCTG-TCCTTC--C--C-----CGA-GG--GCTTCAAGTGGGAGCG-CGTGATGAACTTCGAGGACGGCGGCGTGGTG-ACCG--T-GA-C-CCAGGAC-TC--CTCCCTGCAGGACGGCGAGTTCATCTACAAGGTG---AAGCTGCGCGGCACCAACTTCCCCT-CCGACGGCCCCGTA-ATGCA-GAAGAAGACCATGGGCTG--GGA-GGCCTCCTCCGAGCGGATGTACCCCGAGGA-CGGCGCC-CTGAAGGGCGAGATCAAGCAGA-GGCTGAAGC-TGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACA-AGGCCAAGAAG-CCCGTGCAGCTGCCCGGC-GCCTACAACGTCAACATCAAGT-TG----GA-CATCACCTCCCACAACGAGGA-CTAC-A-C-CA---T-C-G-TGGAACAGTACG-AACGCGCCGAGGGCCGCCACTCCAC-CGGCGGCATGGACGAGCTGTACAAG</th></tr><tr><th>y:</th><th>ATGGTGAGCAAGGGCGAGGA-G----C-T-G--TTCA-C-CGG-GGTGGTGCCCATCCTGGT-CGAGC-TGGACGGCGACGTAAACGGCCACAAGTTC-AG--CGTGTCCGGCGAGGGCGAGGGCGATGCCACCTAC---GGCAAGCTGACC-CTGAAG-TTCATTTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCC-AC-CCTCGTGACCACCCTGACCTACGGCGTGCAGTGC-T-TCAGCCGCTACCCCGACC-ACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGC-GCACCATCTTCTTCAAGGACGACGGCAACTACAAGA-CCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGC-A--ACATC--C-TGGGGCACAAGCTG-G-AGTA-CAACTACAACAGCC-ACAACGTC-TATAT-CATG--GCCGA-CAA--GCAGAAGAACGG-CA--T-C-A-AGG-TGAACTTC-AAGATC--CGCCAC--AA---C---ATCGAG--GACGGC---AGCGTGCAGCTCGCCGACCACTACCA-GC--A-G--AACACC-CC--CATCGGCGACG--GCCCCGTGCTGCTGCCCGACAACC-ACTACCTGAGCACCCAGTCCGCCCTGAGCAA-A-GACCC-CAACGAGAAGC-GCGATCACATGGTCCTGCTGG---AGTTCGTGAC-CGCC----GCCGGGA-T-CACTC-TCGGCATGGACGAGCTGTACAAG</th></tr></table>")

x:,ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGC-A-CATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAG---GGCGAGGGCGAGGGC--CGCC-CCTACGAGGGCACCCAGACCGC-CAAGCTGAAGGTG-ACCA-AGG---G-TGGCC---CCCT-GCCCTTCGCCT-GGGA-CATCCTGTCC--C--C-T-CAGTTCATGT-A-CGGCT-CCAAGGCCTACGTG-A--AGCAC--C--C--C--G-CCGACATCCCCG-A--CTAC-T--TGAAGCTG-TCCTTC--C--C-----CGA-GG--GCTTCAAGTGGGAGCG-CGTGATGAACTTCGAGGACGGCGGCGTGGTG-ACCG--T-GA-C-CCAGGAC-TC--CTCCCTGCAGGACGGCGAGTTCATCTACAAGGTG---AAGCTGCGCGGCACCAACTTCCCCT-CCGACGGCCCCGTA-ATGCA-GAAGAAGACCATGGGCTG--GGA-GGCCTCCTCCGAGCGGATGTACCCCGAGGA-CGGCGCC-CTGAAGGGCGAGATCAAGCAGA-GGCTGAAGC-TGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACA-AGGCCAAGAAG-CCCGTGCAGCTGCCCGGC-GCCTACAACGTCAACATCAAGT-TG----GA-CATCACCTCCCACAACGAGGA-CTAC-A-C-CA---T-C-G-TGGAACAGTACG-AACGCGCCGAGGGCCGCCACTCCAC-CGGCGGCATGGACGAGCTGTACAAG
y:,ATGGTGAGCAAGGGCGAGGA-G----C-T-G--TTCA-C-CGG-GGTGGTGCCCATCCTGGT-CGAGC-TGGACGGCGACGTAAACGGCCACAAGTTC-AG--CGTGTCCGGCGAGGGCGAGGGCGATGCCACCTAC---GGCAAGCTGACC-CTGAAG-TTCATTTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCC-AC-CCTCGTGACCACCCTGACCTACGGCGTGCAGTGC-T-TCAGCCGCTACCCCGACC-ACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGC-GCACCATCTTCTTCAAGGACGACGGCAACTACAAGA-CCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGC-A--ACATC--C-TGGGGCACAAGCTG-G-AGTA-CAACTACAACAGCC-ACAACGTC-TATAT-CATG--GCCGA-CAA--GCAGAAGAACGG-CA--T-C-A-AGG-TGAACTTC-AAGATC--CGCCAC--AA---C---ATCGAG--GACGGC---AGCGTGCAGCTCGCCGACCACTACCA-GC--A-G--AACACC-CC--CATCGGCGACG--GCCCCGTGCTGCTGCCCGACAACC-ACTACCTGAGCACCCAGTCCGCCCTGAGCAA-A-GACCC-CAACGAGAAGC-GCGATCACATGGTCCTGCTGG---AGTTCGTGAC-CGCC----GCCGGGA-T-CACTC-TCGGCATGGACGAGCTGTACAAG


# Exercice 3 : Distribution des scores d’alignement pour des séquences aléatoires

Pour tester si un alignement reflète une réelle similarité biologique, on va évaluer la distribution des scores d’alignement pour des paires de séquences aléatoires.

Q1. En considérant deux séquences aléatoires de même taille N, où chaque nucléotide apparaît avec une probabilité uniforme de ¼, calculer le score moyen attendu pour une superposition sans trou dans le cas où une identité vaut +1 et une différence vaut 0.

```markdown
1/4
```

Q2. La question précédente peut se resoudre analytiquement car on ne considère pas de trou. Pour étendre le résultat precedent à un alignement avec trous, on va se baser sur la simulation de séquences aleatoires.

Générez $R$ paires de séquences aléatoires  de tailles $N$ avec des probabilitées uniformes d'apparition de nucléotides $p_A = p_T = p_G = p_C = $ ¼. Affichez sous forme de violinplots les distribution des scores d'alignements entre chaque paire, obtenu par :
  1. un alignement sans trou (cf. Q1) ;
  2. un alignement local via Smith-Waterman (utilisez le code de l'exercice précédent)

Utilisez le schéma d'évaluation suivant :

In [ ]:
rmap = {"A": 0, "T": 1, "G": 2, "C": 3}
sigma = np.array([[1, -0.5, -0.5, -0.5],
                  [-0.5, 1, -0.5, -0.5],
                  [-0.5, -0.5, 1, -0.5],
                  [-0.5, -0.5, -0.5, 1]])
go =0
ge = 0.5

Q3. Qu'observez-vous ?

```markdown
Votre réponse ici
```

Q4. Quelle conclusion peut-on en tirer sur la significativité d'un alignement ?

```markdown
Votre réponse ici
```